In [1]:
#LaTonya Carter
#import sf3s
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as keras
import sklearn
from sklearn import*
import matplotlib.pyplot as plt
## for data
import pandas as pd
import collections
import json
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud
## for text processing
import re
import nltk
## for language detection
import langdetect 
## for sentiment
from textblob import TextBlob
## for ner
import spacy
## for vectorizer
from sklearn import feature_extraction, manifold
## for word embedding
import gensim.downloader as gensim_api
## for topic modeling
import gensim
import os 
import glob
from zipfile import ZipFile

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', 100000)
pd.set_option('display.max_row', 1000000)
import vaex

from tqdm import  tqdm_notebook as tqdm


In [2]:
!pwd

/Users/ellemafa/Desktop/GTDS/FRL


In [9]:
# Set up the list of zip files to be opened and converted
zip_list = np.sort(np.array(glob.glob('./*.zip')))[::-1]

# The output directory
output_dir = './Users/ellemafa/Desktop/GTDS/FRL/hdf5/'


In [10]:
zip_list

array(['./news_cleaned_2018_02_13.zip'], dtype='<U29')

In [3]:
# for each loaded dataset you have to specify the formats to make DataFrame efficient


dtypes = {'domain' : 'category',
          'type' : 'category', 
          'content' : 'string', 
          'title' :'string'}
columns = ['domain','type' , 'content' ,'title']
#df = pd.read_csv('/Users/ellemafa/Desktop/GTDS/FRL/news_cleaned_2018_02_13.csv', usecols=['domain',  'authors' ,'type' , 'content' ,'title'], dtype=dtypes)

In [4]:
%%time
data_chunk=pd.read_csv('/Users/ellemafa/Desktop/GTDS/FRL/news_cleaned_2018_02_13.csv', sep=',', lineterminator='\n', usecols=['domain' ,'type' , 'content' ,'title'], dtype=dtypes,chunksize=100000)
data_chunk

CPU times: user 2.06 ms, sys: 1.81 ms, total: 3.86 ms
Wall time: 4.12 ms


In [5]:
%%time
#storing the chunks after iterating from chunk object

chunk_data=[chunk for chunk in data_chunk]

#concatnating dataframes to make it a complete dataset

dataset=pd.concat(chunk_data)

CPU times: user 2min 54s, sys: 1min 4s, total: 3min 58s
Wall time: 4min 16s


In [ ]:
len(dataset)

8528956

In [ ]:
dataset.head()

,domain,type,content,title
0,express.co.uk,rumor,"Life is an illusion, at least on a quantum lev...",Is life an ILLUSION? Researchers prove 'realit...
1,barenakedislam.com,hate,"Unfortunately, he hasn’t yet attacked her for ...",Donald Trump
2,barenakedislam.com,hate,The Los Angeles Police Department has been den...,Donald Trump
3,barenakedislam.com,hate,The White House has decided to quietly withdra...,"MORE WINNING! Israeli intelligence source, DEB..."
4,barenakedislam.com,hate,“The time has come to cut off the tongues of t...,"“Oh, Trump, you coward, you just wait, we will..."


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8528956 entries, 0 to 8528955
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   domain   object
 1   type     object
 2   content  string
 3   title    string
dtypes: object(2), string(2)
memory usage: 260.3+ MB


In [ ]:
dataset.domain.value_counts()

nytimes.com                              1542685
beforeitsnews.com                         781009
dailykos.com                              660626
express.co.uk                             477264
nationalreview.com                        319556
sputniknews.com                           318745
abovetopsecret.com                        238622
wikileaks.org                             199031
www.newsmax.com                           107420
pravda.ru                                  95141
www.ammoland.com                           92846
wikispooks.com                             80683
lifezette.com                              76999
investmentwatchblog.com                    68559
dailycaller.com                            66706
breitbart.com                              60436
thepostemail.com                           59236
rawstory.com                               58659
naturalnews.com                            56383
pjmedia.com                                52642
thedailysheeple.com 

### How many domains per type?

In [7]:
dataset.groupby('type')['domain'].nunique().reset_index()

,type,domain
0,bias,95
1,clickbait,21
2,conspiracy,84
3,fake,107
4,hate,15
5,junksci,24
6,political,55
7,reliable,80
8,rumor,5
9,satire,78


### Count of types per domain

In [16]:
dataset.pivot_table(index='domain', 
               columns='type',
               aggfunc='size',
               fill_value=0)

INFO:MainThread:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:MainThread:numexpr.utils:NumExpr defaulting to 8 threads.


type,bias,clickbait,conspiracy,fake,hate,junksci,political,reliable,rumor,satire,unknown,unreliable
domain,,,,,,,,,,,,
100percentfedup.com,97,0,0,0,0,0,0,0,0,0,0,0
16wmpo.com,0,0,0,1,0,0,0,0,0,0,0,0
21stcenturywire.com,0,0,19897,0,0,0,0,0,0,0,0,0
4threvolutionarywar.wordpress.com,5394,0,0,0,0,0,0,0,0,0,0,0
70news.wordpress.com,0,0,0,2294,0,0,0,0,0,0,0,0
82.221.129.208,0,0,32,0,0,0,0,0,0,0,0,0
Newsmax.com,0,0,0,0,0,0,0,0,0,0,0,1
NoMoreFakeNews.com,0,0,2,0,0,0,0,0,0,0,0,0
Zengardner.com,0,0,0,0,0,0,0,0,0,0,1,0


### Down-sampling involves randomly removing observations from the majority class to prevent its signal from dominating the learning algorithm.

The most common heuristic for doing so is resampling without replacement.

The process is similar to that of up-sampling. Here are the steps:

First, we'll separate observations from each class into different DataFrames.
Next, we'll resample the majority class without replacement, setting the number of samples to match that of the minority class.
Finally, we'll combine the down-sampled majority class DataFrame with the original minority class DataFrame.

In [8]:
#copy orginal import
#df_exp = df.copy()
#keep all row that have fake/reliable "type"
df_exp = dataset[dataset['type'].isin(['fake', 'reliable'])]

In [9]:
len(df_exp)

2807968

In [10]:
df_exp.type.value_counts()

reliable    1913222
fake         894746
Name: type, dtype: int64

In [25]:
from sklearn.utils import resample


#Separate majority and minority classes
df_majority = df_exp[df_exp.type=="reliable"]
df_minority = df_exp[df_exp.type=="fake"]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=894746,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.type.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

fake        894746
reliable    894746
Name: type, dtype: int64

In [13]:
df_downsampled.groupby('type')['domain'].nunique().reset_index()

,type,domain
0,fake,107
1,reliable,80


In [15]:
df_downsampled.pivot_table(index='domain', 
               columns='type',
               aggfunc='size',
               fill_value=0)

type,fake,reliable
domain,,
16wmpo.com,1,0
70news.wordpress.com,2294,0
abcnews.go.com,0,4175
af.reuters.com,0,2488
americannews.com,197,0
americanoverlook.com,47,0
anonjekloy.tk,38,0
au.be.yahoo.com,0,340
au.finance.yahoo.com,0,507


In [30]:
df_downsampled.head()

,domain,type,content,title
0,people.com,reliable,"Hilary Duff/Instagram If you’ve got it, flaun...",Hilary Duff Flaunts Her Bikini Bod While Vacat...
1,www.npr.org,reliable,Enlarge this image This red squirrel has the d...,Study Finds That Leprosy Circulates In Britain...
2,www.latimes.com,reliable,Ski Butlers rental delivery service expands to...,Ski Butlers rental delivery service expands to...
3,af.reuters.com,reliable,CORRECTED-AFDB to lend Tunisia up to 2 bln eur...,CORRECTED-AFDB to lend Tunisia up to 2 bln eur...
4,nytimes.com,reliable,With the N.C.A.A. investigating allegations th...,Fighting Words


In [29]:
df_downsampled = df_downsampled.reset_index(drop=True)

# NLP Preproccessing - Clean Content Field

Data Cleaning
Once we collected the data, we had to pre-process it in order to construct a corpus for our sentiment analysis and word cloud creation. We performed the following cleaning measures:

Change all tweets to lowercase

Remove all stopwords (as defined by nltk)

Remove mentions, links, and hashtags

Remove all punctuation and brackets

Remove all one-character words

In [16]:
import os
import re
import nltk
import pandas as pd
import numpy as np
from afinn import Afinn
import matplotlib.pyplot as plt
from bokeh.models import HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Tabs
from bokeh.layouts import gridplot, column, row, WidgetBox
from bokeh.plotting import figure, show, output_file
from collections import Counter
import networkx as nx
from netwulf import visualize

In [17]:
import nltk
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ellemafa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
print(nltk.corpus.stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [20]:
stopwords = nltk.corpus.stopwords.words("english")

# Clean tweets for sentiment analysis
def get_clean_words(words):
    def _isnum(w):
        try:
            int(w)
            return True
        except ValueError:
            return False
        
    # Set words to lowercase and remove them if they are stop words
    words = [w.lower() for w in words if w.lower() not in stopwords]

    # Remove punctuation
    words = [w.replace('(', '') for w in words]
    words = [w.replace(')', '') for w in words]
    words = [w.replace('?', '') for w in words]
    words = [w.replace(',', '') for w in words]
    words = [w.replace('.', '') for w in words]
    words = [w.replace('"', '') for w in words]
    words = [w.replace('!', '') for w in words]
    words = [w.replace(':', '') for w in words]
    words = [w.replace('&amp;', '') for w in words]
    words = [w.replace('/', '') for w in words]
    words = [w.replace('[', '') for w in words]
    words = [w.replace(']', '') for w in words]
    
    # Remove numbers
    words = [w for w in words if not _isnum(w)]
    
    # Remove links
    words = [w for w in words if 'http' not in w]
        
    # Remove hashtags
    words = [w for w in words if not w.startswith("#")]
    
    # Remove mentions
    words = [w for w in words if not w.startswith("@")]
    
    # Only keep words with more than one character
    words = [w for w in words if len(w) > 1]    
    w = " ".join(words)
    return w

In [32]:
 #Funtion to pass each tweet into the clean words function and create a new list
def clean_content(df):
    new_text = []
    for i in range(len(df)):
        new_text.append(get_clean_words(df_downsampled['content'].values[i].split()))
    df_downsampled.content = new_text

In [33]:
clean_content(df_downsampled)

In [41]:
df_downsampled.head(10)

,domain,type,content,title
0,people.com,reliable,hilary duffinstagram you’ve got it flaunt it h...,Hilary Duff Flaunts Her Bikini Bod While Vacat...
1,www.npr.org,reliable,enlarge image red squirrel disfiguring sores l...,Study Finds That Leprosy Circulates In Britain...
2,www.latimes.com,reliable,ski butlers rental delivery service expands re...,Ski Butlers rental delivery service expands to...
3,af.reuters.com,reliable,corrected-afdb lend tunisia bln euros next fiv...,CORRECTED-AFDB to lend Tunisia up to 2 bln eur...
4,nytimes.com,reliable,ncaa investigating allegations agents gave imp...,Fighting Words
5,www.legacy.com,reliable,obituary condolences wright kyle dkyle douglas...,"Wright, Kyle D."
6,nytimes.com,reliable,segal-steve ''no mr nice guy'' sad loss is str...,"Paid Notice: Deaths SEGAL, STEVE"
7,nytimes.com,reliable,spring dozen kids surrounding settlements join...,The Unsettlers
8,nytimes.com,reliable,amos hyperbolic coming bowl hometown justin le...,Life at the Bottom of the Bowl Pyramid
9,nytimes.com,reliable,fort greene co-op $300000 brooklyn clinton ave...,On The Market


In [36]:
df_downsampled.to_csv('df_downsampled.csv')

## Affinity Score - Sentiment Analyis - Content Column

In [59]:
# Function to find the affinity score of a list of tweets
afinn = Afinn()

def get_affinity_scores(tweets):
    scores = []
    count = 0
    for t in tweets:
        if len(t) > 0:
            scores.append(afinn.score(t) / len(t))
        else:
            count += 1
            scores.append(0)
    return scores

In [61]:
from afinn import Afinn

In [60]:
new_affin = get_affinity_scores(df_downsampled['content'].tolist())

In [77]:
df_downsampled['title']= df_downsampled['title'].astype(str)

In [76]:
df_downsampled['title'].values

<StringArray>
[                    'Hilary Duff Flaunts Her Bikini Bod While Vacationing with Trainer Boyfriend Jason Walsh',
                             "Study Finds That Leprosy Circulates In Britain's Red Squirrels : Goats and Soda",
                                       'Ski Butlers rental delivery service expands to 37 resorts in the West',
                             'CORRECTED-AFDB to lend Tunisia up to 2 bln euros over next five years -official',
                                                                                              'Fighting Words',
                                                                                             'Wright, Kyle D.',
                                                                           'Paid Notice: Deaths  SEGAL, STEVE',
                                                                                              'The Unsettlers',
                                                                      'Life at the Bottom 

In [68]:
df_downsampled['content_affin'] = new_affin

In [65]:
type(new_affin)

list

In [67]:
df_affin_content = pd.DataFrame(data={"content_affin_score": new_affin})
df_affin_content.to_csv("./content_affin.csv", sep=',',index=False)

In [78]:
len(df_affin_content)

1789492

In [80]:
df_affin_content.describe()

,content_affin_score
count,1.789492e+06
mean,2.357004e-03
std,1.769268e-02
min,-3.076923e-01
25%,-5.257624e-03
50%,3.112033e-03
75%,1.096121e-02
max,6.153846e-01


In [81]:
df_downsampled.head()

,domain,type,content,title,content_affin
0,people.com,reliable,hilary duffinstagram you’ve got it flaunt it h...,Hilary Duff Flaunts Her Bikini Bod While Vacat...,0.015023
1,www.npr.org,reliable,enlarge image red squirrel disfiguring sores l...,Study Finds That Leprosy Circulates In Britain...,-0.003860
2,www.latimes.com,reliable,ski butlers rental delivery service expands re...,Ski Butlers rental delivery service expands to...,0.005756
3,af.reuters.com,reliable,corrected-afdb lend tunisia bln euros next fiv...,CORRECTED-AFDB to lend Tunisia up to 2 bln eur...,-0.000751
4,nytimes.com,reliable,ncaa investigating allegations agents gave imp...,Fighting Words,0.002381


In [82]:
df_downsampled.to_csv("./df_content_affin.csv", sep=',',index=False)